In [7]:
import django_initializer
from api.models.user.user import User
from django.contrib.auth.models import User as AuthUser
from json import loads, dumps
import json
import lib
import re
import service
from api.models.campaign.campaign import Campaign
from api.models.user.user_subscription import UserSubscription
from service.web_crawler.facebook_shared_list_crawler import FacebookSharedListCrawler
import pickle
import platform
import time
from typing import OrderedDict
import functools, logging, traceback
from api import models
import service
from django.conf import settings
import os
import random
import lib
import arrow
from service.web_crawler.facebook_crawler import FacebookCrawler
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
def to_dict(input_ordered_dict):
    return loads(dumps(input_ordered_dict))

In [9]:
campaign_lucky_draw_id = "1505"
lucky_draw = lib.util.verify.Verify.get_lucky_draw(campaign_lucky_draw_id)
username = lucky_draw.campaign.facebook_page.username
post_id = lucky_draw.campaign.facebook_campaign.get("post_id", "")
fb_crawler = FacebookSharedListCrawler(username, post_id, open_browser=True)
shared_user_name_set = fb_crawler.start()
print(shared_user_name_set)

remote
login with cookies
validate_user
check if Go to App button shows up
Traceback (most recent call last):
  File "/home/liveshowseller/service/web_crawler/facebook_shared_list_crawler.py", line 73, in validate_user
    go_to_app_button = self.driver.find_element(By.CSS_SELECTOR, "button[value='Go to App']")
  File "/root/.cache/pypoetry/virtualenvs/liveshowseller-4MbNXGbY-py3.9/lib/python3.9/site-packages/selenium/webdriver/remote/webdriver.py", line 856, in find_element
    return self.execute(Command.FIND_ELEMENT, {
  File "/root/.cache/pypoetry/virtualenvs/liveshowseller-4MbNXGbY-py3.9/lib/python3.9/site-packages/selenium/webdriver/remote/webdriver.py", line 429, in execute
    self.error_handler.check_response(response)
  File "/root/.cache/pypoetry/virtualenvs/liveshowseller-4MbNXGbY-py3.9/lib/python3.9/site-packages/selenium/webdriver/remote/errorhandler.py", line 243, in check_response
    raise exception_class(message, screen, stacktrace)
selenium.common.exceptions.NoSuchEl

In [4]:
page_username = "kuai.mai.2021"
target_post_id = "517999680166258"
language = "zh_hant"
langauag_map = {
    "zh_hant": "中文(台灣)",
    "en": "English (US)",
    "zh_hans": "中文(简体)",
    "vi": "Tiếng Việt"
}
fb_crawler = FacebookSharedListCrawler(page_username, target_post_id, lang=language)

In [ ]:
data = fb_crawler.start()
print(data)


In [6]:
fb_crawler.quit_driver()

In [4]:
class RenewFacebookCookiesCrawler(FacebookCrawler):
    
        
    def login(self, validate=False):
        print("login")
        count = 0
        while not self.is_login and count <= 4:
            if not validate:
                self.driver.get(self.login_url)
            email_input = self.driver.find_element(By.ID, "m_login_email")
            pass_input = self.driver.find_element(By.ID, "m_login_password")
            login_button = self.driver.find_element(By.NAME, "login")
            self.actions.send_keys_to_element(email_input, self.email)
            self.actions.send_keys_to_element(pass_input, self.password)
            self.actions.click(login_button)
            self.actions.perform()
            count += 1
            self.is_login = True
        time.sleep(2)
        self.validate_user()
        self.save_cookies()
        
        
    def validate_user(self):
        print("validate_user")
        # check if page pops up log in with one tap
        try:
            self.driver.find_element(By.CSS_SELECTOR, "input[value='regular_login']")
            ok_button = self.driver.find_element(By.CSS_SELECTOR, "button[value='OK']")
            self.actions.click(ok_button).perform()
            self.save_login = True
        except:
            print(traceback.format_exc())
            
        if not self.save_login:
            # check new page identity double check
            try:
                my_profile = self.driver.find_element(By.CSS_SELECTOR, f"div[role='button']")
                self.actions.click(my_profile).perform()

                pass_input = self.wait.until(
                    EC.presence_of_element_located((By.NAME, "pass"))
                )
                self.actions.send_keys_to_element(pass_input, self.password)
                login_button = self.driver.find_element(By.CSS_SELECTOR , "button[type='submit']")
                self.actions.click(login_button).perform()
                self.save_login = True
            except:
                print(traceback.format_exc())
            
        if not self.save_login:
            # check header identity double check
            try:
                login_button = self.driver.find_element(By.CSS_SELECTOR , "button[type='submit']")
                self.actions.click(login_button).perform()
                pass_input = self.wait.until(
                    EC.presence_of_element_located((By.NAME, "pass"))
                )
                self.actions.send_keys_to_element(pass_input, self.password)
                login_button = self.driver.find_element(By.CSS_SELECTOR , "button[type='submit']")
                self.actions.click(login_button).perform()
                self.save_login = True
            except:
                print(traceback.format_exc())
    
    def start(self):
        try:
            self.create_driver()
            self.create_wait_object()
            self.create_action_object()
            self.login()
            self.driver.quit()
        except Exception as e:
            print(traceback.format_exc())
            if self.driver:
                self.driver.quit()

In [5]:
crawler = RenewFacebookCookiesCrawler(open_browser=True)
crawler.start()

remote
login
validate_user
save facebook cookies to /home/liveshowseller/pkl/cookies/facebook_cookie.pkl


In [8]:
class FacebookSharedListCrawler(FacebookCrawler):
    def __init__(self, page_username, target_post_id, lang="en", chrome_driver_path=os.path.join(settings.BASE_DIR, 'chromedriver.exe'), open_browser=False):
        super().__init__(open_browser)
        self.page_url = f'https://m.facebook.com/{page_username}/'
        self.target_post_id = target_post_id
        self.post_reference_id = ""
        self.post_shared_list_url = "https://m.facebook.com/browse/shares?id="
        
        self.lang_map = {
            "zh_hant": "中文(台灣)",
            "en": "English (US)",
            "zh_hans": "中文(简体)",
            "vi": "Tiếng Việt"
        }
        self.lang = self.lang_map.get(lang)
    
    
    
    def login_with_cookies(self, cookies):
        print("login with cookies")
        for c in cookies:
            self.driver.add_cookie(c)
        self.is_login = True
        self.driver.refresh()
        self.validate_user()
        
    def login(self, validate=False):
        print("login")
        count = 0
        while not self.is_login and count <= 4:
            if not validate:
                self.driver.get(self.login_url)
            email_input = self.driver.find_element(By.ID, "m_login_email")
            pass_input = self.driver.find_element(By.ID, "m_login_password")
            login_button = self.driver.find_element(By.NAME, "login")
            self.actions.send_keys_to_element(email_input, self.email)
            self.actions.send_keys_to_element(pass_input, self.password)
            self.actions.click(login_button)
            self.actions.perform()
            count += 1
            self.is_login = True
        time.sleep(2)
        self.validate_user()
        self.save_cookies()
    
    def validate_user(self):
        print("validate_user")
        # check if page pops up log in with one tap
        try:
            print("check if Go to App button shows up")
            # check if Go to App button shows up, if yes, pass validation
            go_to_app_button = self.driver.find_element(By.CSS_SELECTOR, "button[value='Go to App']")
            if go_to_app_button:
                self.save_login = True
        except:
            print(traceback.format_exc())
        
        if not self.save_login:
            try:
                self.driver.find_element(By.CSS_SELECTOR, "input[value='regular_login']")
                ok_button = self.driver.find_element(By.CSS_SELECTOR, "button[value='OK']")
                if ok_button:
                    self.actions.click(ok_button).perform()
                    self.save_login = True
            except:
                print(traceback.format_exc())
            
        if not self.save_login:
            # check new page identity double check
            print("new page identity double check")
            try:
                my_profile = self.driver.find_element(By.CSS_SELECTOR, f"div[role='button']")
                self.actions.click(my_profile).perform()

                pass_input = self.wait.until(
                    EC.presence_of_element_located((By.NAME, "pass"))
                )
                self.actions.send_keys_to_element(pass_input, self.password)
                login_button = self.driver.find_element(By.CSS_SELECTOR , "button[type='submit']")
                if login_button:
                    self.actions.click(login_button).perform()
                    self.save_login = True
            except:
                print(traceback.format_exc())
            
        if not self.save_login:
            # check header identity double check
            print("header identity double check")
            try:
                login_button = self.driver.find_element(By.CSS_SELECTOR , "button[type='submit']")
                self.actions.click(login_button).perform()
                pass_input = self.wait.until(
                    EC.presence_of_element_located((By.NAME, "pass"))
                )
                self.actions.send_keys_to_element(pass_input, self.password)
                login_button = self.driver.find_element(By.CSS_SELECTOR , "button[type='submit']")
                if login_button:
                    self.actions.click(login_button).perform()
                    self.save_login = True
            except:
                print(traceback.format_exc())
            
    def switch_language(self):
        self.driver.get("https://m.facebook.com/language/")
        current_language = self.driver.find_element(By.CLASS_NAME, "_5551")
        
    def get_post_shared_list_url(self):
        found = False
        count = 0
        while not found:
            self.driver.execute_script("window.scrollTo(0,document.body.scrollHeight);")
            article = self.wait.until(
                    EC.visibility_of_element_located((By.TAG_NAME, "article"))
                )
            if article:
                articles = self.driver.find_elements(By.XPATH, "//article")
               
                for article in articles:
                    if self.post_reference_id:
                        break
                    video_data = json.loads(article.get_attribute("data-ft"))
                    if video_data.get('video_id', None) != self.target_post_id:
                        continue
                    print(video_data)
                    post_data = json.loads(article.get_attribute("data-store"))
                    self.post_reference_id = str(post_data.get("feedback_target",""))
                    found = True
#                     for key,value in post_data.items():
#                         if key == "share_id":
# #                             print(key)
# #                             print(value)
#                             if ":" not in str(value):
#                                 post_id = str(value)
#                             else:
#                                 post_id = value.split(":")[1]
#                             if post_id != self.target_post_id:
#                                 break

#                         if key == "feedback_target":
# #                             print(key)
# #                             print(value)
#                             self.post_reference_id = str(value)
#                             break
            if count == 20:
                found = True
            count += 1
        print(self.post_shared_list_url + self.post_reference_id)
        return self.post_shared_list_url + self.post_reference_id
        
    def expand_all(self):
        try:
            while True:
                see_more_button = self.driver.find_element(By.ID, "m_more_item")
                if see_more_button:
                    self.actions.click(see_more_button).perform()
        except NoSuchElementException:
            print("Element is not present")
            
    def collect_names(self):
        names = set([el.text for el in self.driver.find_elements(By.TAG_NAME, "strong")])
        return names
    
    def start(self):
        start = arrow.now()
        try:
            self.create_driver()
            self.create_wait_object()
            self.create_action_object()
            
            cookies = self.find_cookies()
            if cookies:
                self.driver.get(self.page_url)
                self.login_with_cookies(cookies)
            else:    
                self.login()
                self.driver.get(self.page_url)
                
            post_shared_list_url = self.get_post_shared_list_url()
            self.driver.get(post_shared_list_url)
            
            if not self.save_login:
                self.validate_user()
            self.expand_all()
            name_list = self.collect_names()
            end = arrow.now()
            print(f"Spent: {end - start}")
            self.driver.quit()
            return name_list
        except Exception as e:
            print(traceback.format_exc())
            if self.driver:
                self.driver.quit()
            return {}

In [ ]:
print("start crawler")
campaign_lucky_draw_id = "1505"
lucky_draw = lib.util.verify.Verify.get_lucky_draw(campaign_lucky_draw_id)
username = lucky_draw.campaign.facebook_page.username
post_id = lucky_draw.campaign.facebook_campaign.get("post_id", "")
fb_crawler = FacebookSharedListCrawler(username, post_id, open_browser=True)
shared_user_name_set = fb_crawler.start()
print(shared_user_name_set)

start crawler
remote
login with cookies
validate_user
check if Go to App button shows up
{'qid': '-5328316083792569096', 'mf_story_key': '809274420379906', 'top_level_post_id': '809274420379906', 'tl_objid': '809274420379906', 'content_owner_id_new': '100063671276299', 'throwback_story_fbid': '809274420379906', 'page_id': '109130787659110', 'story_location': 4, 'story_attachment_style': 'video_inline', 'ott': 'ottAX_0uHhBirslVfel', 'video_id': '809274420379906', 'sty': 38, 'ent_attachement_type': 'VideoAttachment', 'app_id': '6628568379', 'page_insights': {'100063671276299': {'page_id': '100063671276299', 'page_id_type': 'page', 'actor_id': '100063671276299', 'dm': {'isShare': 0, 'originalPostOwnerID': 0}, 'psn': 'EntVideoCreationStory', 'post_context': {'object_fbtype': 1, 'publish_time': 1666172309, 'story_name': 'EntVideoCreationStory', 'story_fbid': ['809274420379906']}, 'role': 1, 'sl': 4, 'targets': [{'actor_id': '100063671276299', 'page_id': '100063671276299', 'post_id': '809274

In [ ]:
driver = webdriver.Remote(command_executor='http://localhost:4444', options=self.chrome_options)

In [3]:
crawler = FacebookCrawler()
driver = crawler.create_driver()

remote


ProtocolError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))